#### 3.1 Supervised DA

##### 3.1.1 In-domain fine-tuning

Подготовка данных

In [2]:
import datasets
import typing as tp
import inspect

conll = datasets.load_dataset("conll2003") # DatasetDict
wnut = datasets.load_dataset("wnut_17")

conll.save_to_disk("datasets/conll2003")
wnut.save_to_disk("datasets/wnut_17")

CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names # всего 9 тегов
WNUT_NER_TAGS = wnut['train'].features['ner_tags'].feature.names

Saving the dataset (0/1 shards):   0%|          | 0/14041 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3250 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3453 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3394 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1009 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1287 [00:00<?, ? examples/s]

In [3]:
conll

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
wnut

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [4]:
CONLL_NER_TAGS

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [5]:
WNUT_NER_TAGS

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [6]:
label_mapping = {
    'O': 'O',
    'B-location': 'B-LOC',
    'I-location': 'I-LOC',
    'B-group': 'B-ORG',
    'B-corporation': 'B-ORG',
    'B-person': 'B-PER',
    'B-creative-work': 'B-MISC',
    'B-product': 'B-MISC',
    'I-person': 'I-PER',
    'I-creative-work': 'I-MISC',
    'I-corporation': 'I-ORG',
    'I-group': 'I-ORG',
    'I-product': 'I-MISC'
}

labelindexmapping = {WNUT_NER_TAGS.index(k):CONLL_NER_TAGS.index(v) for k, v in label_mapping.items()}

import data_prep

converted_wnut = wnut.map(lambda x: data_prep.convert_label_sequence(x, labelindexmapping))

import numpy as np
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline

import torch 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

from tqdm.auto import tqdm

from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)

model_name = "dslim/bert-base-NER"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
from datasets import load_dataset
import torch
from tqdm.auto import tqdm
from transformers import AdamW, BertTokenizer, BertForMaskedLM, set_seed
import pandas as pd
import warnings

set_seed(42)
warnings.filterwarnings("ignore")
import data_prep
importlib.reload(data_prep)

label2id_conll= {'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8
}

id2label_conll = {v : k for k, v in label2id_conll.items()}

test_sentence = "His name is Jerry Abrahamson"
test_example = {"tokens": test_sentence.split(" "), "ner_tags": [0, 0, 0, 1, 2]}
# test_result = data_prep.tokenize_and_preserve_tags(test_example, tokenizer, model.config.label2id)
test_result = data_prep.tokenize_and_preserve_tags(test_example, tokenizer, label2id_conll)

assert tokenizer.decode(test_result['input_ids']) == '[CLS] His name is Jerry Abrahamson [SEP]'

                                     #CLS     His  name is    Jerry    Abraham   ##son      SEP
assert test_result['text_labels'] == ['O'] + ["O", "O", "O", "B-PER", "I-PER",  "I-PER"] + ["O"]

conll = conll.map(lambda x: data_prep.tokenize_and_preserve_tags(x, tokenizer, label2id_conll))

wnut = converted_wnut
wnut = wnut.map(lambda x: data_prep.tokenize_and_preserve_tags(x, tokenizer, label2id_conll))

conll.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], output_all_columns=True)
wnut.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], output_all_columns=True)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
import importlib
import data_prep
importlib.reload(data_prep)

test_sentence = "His name is Jerry Abrahamson"
test_example = {"tokens": test_sentence.split(" "), "ner_tags": [0, 0, 0, 1, 2]}
test_result = data_prep.tokenize_and_preserve_tags(test_example, tokenizer, label2id_conll)
print(test_result.word_ids())
print(test_result['input_ids'])
print([tokenizer.decode(token) for token in test_result['input_ids']])

[None, 0, 1, 2, 3, 4, 4, None]
[101, 1230, 1271, 1110, 5466, 7752, 2142, 102]
['[CLS]', 'His', 'name', 'is', 'Jerry', 'Abraham', '##son', '[SEP]']


In [8]:
test_result

{'input_ids': [101, 1230, 1271, 1110, 5466, 7752, 2142, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1], 'tokens': ['His', 'name', 'is', 'Jerry', 'Abrahamson'], 'ner_tags': [0, 0, 0, 1, 2], 'labels': [0, 0, 0, 0, 1, 2, 2, 0], 'text_labels': ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O']}

Теперь есть готовые датасеты conll и wnut

Создадим даталоадеры для трейн и теста conll и wnut

In [9]:
input_keys = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']
batch_size = 16
wnut_train_dataloader = torch.utils.data.DataLoader(wnut["train"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
wnut_test_dataloader = torch.utils.data.DataLoader(wnut["test"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
conll_train_dataloader = torch.utils.data.DataLoader(conll["train"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
conll_test_dataloader = torch.utils.data.DataLoader(conll["test"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))


In [10]:
batch=next(iter(wnut_test_dataloader))
print(len(batch['ner_tags'][7]))
print(len(batch['input_ids'][7]))
print(len(batch['tokens'][7]))
print(len(batch['labels'][7]))

14
45
14
45


In [11]:
test_batch = {k : [v] for k, v in test_result.items()}

Измерим метрики на базовой версии BERTа

In [12]:
model_name = "dslim/bert-base-NER" # дообучена на датасете CoNLL-2003
model_base = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_base.to(device)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [13]:
test_batch

{'input_ids': [[101, 1230, 1271, 1110, 5466, 7752, 2142, 102]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]],
 'tokens': [['His', 'name', 'is', 'Jerry', 'Abrahamson']],
 'ner_tags': [[0, 0, 0, 1, 2]],
 'labels': [[0, 0, 0, 0, 1, 2, 2, 0]],
 'text_labels': [['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O']]}

In [14]:
# тест, что все адекватно предсказывается
import torch
from collections import defaultdict

batched_test_batch = defaultdict(list)

for key, value in test_batch.items():
    try:
        batched_test_batch[key] = torch.tensor(value)
    except ValueError:
        batched_test_batch[key] = value

for key in batched_test_batch:
    if isinstance(batched_test_batch[key], torch.Tensor):
        batched_test_batch[key] = batched_test_batch[key].to(device)

batched_test_batch

defaultdict(list,
            {'input_ids': tensor([[ 101, 1230, 1271, 1110, 5466, 7752, 2142,  102]], device='cuda:0'),
             'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'),
             'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'),
             'tokens': [['His', 'name', 'is', 'Jerry', 'Abrahamson']],
             'ner_tags': tensor([[0, 0, 0, 1, 2]], device='cuda:0'),
             'labels': tensor([[0, 0, 0, 0, 1, 2, 2, 0]], device='cuda:0'),
             'text_labels': [['O',
               'O',
               'O',
               'O',
               'B-PER',
               'I-PER',
               'I-PER',
               'O']]})

In [15]:
model_base.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

In [16]:
id2label_conll

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [17]:
import importlib
import model_utils
importlib.reload(model_utils)

from model_utils import NamedEntityPredictor

ner_test = NamedEntityPredictor(model_base, tokenizer, id2label_conll)
ner_test.predict(batched_test_batch)

{'predicted_labels': [['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O']],
 'loss': tensor(3.5653, device='cuda:0'),
 'logits': tensor([[[ 6.8225, -0.2862, -1.0980, -0.3375, -1.1505, -1.0953, -1.7006,
           -0.8111, -1.3737],
          [ 9.3304, -1.1577, -2.1227, -0.1745, -1.8043, -1.7342, -2.2795,
           -0.7647, -1.3315],
          [ 9.7455, -1.0748, -1.7252, -0.2097, -1.3579, -2.4633, -1.7645,
           -1.0774, -1.2969],
          [ 9.3520, -1.2463, -1.9811, -0.2193, -1.5422, -2.2553, -1.8455,
           -0.9268, -1.1779],
          [-0.8416, -1.2910, -2.3588,  8.7858, -0.5230, -1.6059, -2.6656,
           -0.3291, -1.6418],
          [-0.7083, -1.2668, -1.3766, -1.0028,  8.9504, -1.9978, -0.7555,
           -0.9198, -1.0510],
          [ 1.7984, -1.4755, -1.4079, -0.3893,  6.5783, -2.9142, -1.2547,
           -0.7049, -1.4439],
          [ 4.4946, -0.4864, -1.3231,  1.8384,  0.9670, -3.0517, -1.8221,
           -0.5035, -1.0015]]], device='cuda:0')}

In [18]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, get_scheduler
import torch
from torch.nn.utils import clip_grad_norm_ # выполняет обрезку градиентов для избежания их взрыва при обучении
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np

import importlib
import data_prep
import model_utils
importlib.reload(data_prep)
importlib.reload(model_utils)

from model_utils import train_eval_ner

Файн-тюн сразу всего BERTа

In [61]:
model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
optimizer = torch.optim.AdamW(trainable_params, lr=1e-5, weight_decay=0.01)

num_epochs = 2
warmup_factor = 0.1

ft_train_dataset = wnut['train']
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(warmup_factor * num_training_steps),
    num_training_steps=num_training_steps,
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:03<00:00, 22.89it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 213/213 [00:23<00:00,  8.97it/s]


Train: 0.12357293134074256 	 Test: 0.3769584274218406


Epoch 2: 100%|████████████████████████████████████████████████████████████████████████| 213/213 [00:23<00:00,  8.92it/s]


Train: 0.10321705974638462 	 Test: 0.38193389897545177

Evaluating fine-tuned model


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:03<00:00, 22.52it/s]



                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.84 (+0.09)  0.69 (-0.15)  0.76 (-0.03)  0.30 (-0.15)   
MISC          0.20 (-0.38)  0.66 (-0.04)  0.30 (-0.33)  0.07 (-0.10)   
ORG           0.68 (-0.03)  0.85 (+0.03)  0.75 (-0.01)  0.27 (+0.11)   
PER           0.11 (-0.25)  0.07 (-0.51)  0.08 (-0.37)  0.30 (+0.00)   
micro avg     0.46 (-0.12)  0.56 (-0.18)  0.50 (-0.15)  0.17 (-0.07)   
macro avg     0.46 (-0.14)  0.57 (-0.17)  0.47 (-0.19)  0.24 (-0.03)   
weighted avg  0.50 (-0.11)  0.56 (-0.18)  0.51 (-0.15)  0.24 (-0.02)   

                    rec_ft         f1_ft  
LOC           0.13 (-0.38)  0.18 (-0.30)  
MISC          0.11 (-0.11)  0.09 (-0.10)  
ORG           0.24 (-0.06)  0.25 (+0.04)  
PER           0.05 (-0.25)  0.09 (-0.21)  
micro avg     0.12 (-0.19)  0.14 (-0.13)  
macro avg     0.13 (-0.20)  0.15 (-0.14)  
weighted avg  0.12 (-0.19)  0.14 (-0.14)  

metric |           value
------------------------
f1     |    0.14 (-0.13)
pr

Кажется происходит сильное переобучение под train WNUTа.

Если файн-тюнить только классификатор

In [95]:
model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for param in model_ft.bert.parameters():
    param.requires_grad = False
trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
# optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)
optimizer = torch.optim.Adam(trainable_params, lr=2e-5, betas=(0.9, 0.999), eps=1e-08)

num_epochs = 10
warmup_factor = 0.1

ft_train_dataset = wnut['train']
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(warmup_factor * num_training_steps),
    num_training_steps=num_training_steps,
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:03<00:00, 22.67it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 213/213 [00:07<00:00, 29.75it/s]


...

Epoch: 8	 Train: 0.23220971399839496 	 Test: 0.37771314860861976



Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 213/213 [00:07<00:00, 29.40it/s]


Epoch: 9	 Train: 0.22781684008282674 	 Test: 0.3719683326320884



Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 213/213 [00:07<00:00, 29.39it/s]


Epoch: 10	 Train: 0.22641523799775912 	 Test: 0.37011940050640224

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.76 (+0.01)  0.83 (-0.01)  0.79 (+0.00)  0.50 (+0.05)   
MISC          0.53 (-0.05)  0.67 (-0.03)  0.59 (-0.04)  0.15 (-0.02)   
ORG           0.68 (-0.03)  0.78 (-0.04)  0.73 (-0.03)  0.26 (+0.10)   
PER           0.40 (+0.04)  0.59 (+0.01)  0.48 (+0.03)  0.39 (+0.09)   
micro avg     0.59 (+0.01)  0.73 (-0.01)  0.65 (+0.00)  0.29 (+0.05)   
macro avg     0.59 (-0.01)  0.72 (-0.02)  0.65 (-0.01)  0.32 (+0.05)   
weighted avg  0.61 (+0.00)  0.73 (-0.01)  0.66 (+0.00)  0.32 (+0.06)   

                    rec_ft         f1_ft  
LOC           0.43 (-0.08)  0.47 (-0.01)  
MISC          0.17 (-0.05)  0.16 (-0.03)  
ORG           0.27 (-0.03)  0.26 (+0.05)  
PER           0.26 (-0.04)  0.31 (+0.01)  
micro avg     0.26 (-0.05)  0.28 (+0.01)  
macro avg     0.28 (-0.05)  0.30 (+0.01)  
weighted avg  0.26 (-0.05)  0.28 (+0.00)  

metric |   

In [96]:
ft_metrics_after

{'train_loss': [0.4141742690278331,
  0.3634748491043216,
  0.32342260077315876,
  0.2925367337857054,
  0.2690735067210287,
  0.25196356457513824,
  0.24004674139716817,
  0.23220971399839496,
  0.22781684008282674,
  0.22641523799775912],
 'test_loss': [0.6115067956256278,
  0.550726252021613,
  0.4996400620834327,
  0.4584120437502861,
  0.42674131176353974,
  0.40386411979978465,
  0.38808650256674965,
  0.37771314860861976,
  0.3719683326320884,
  0.37011940050640224],
 'f1': [0.2751986616478461,
  0.2790294627383016,
  0.278391515687141,
  0.28208616780045354,
  0.286509040333797,
  0.2857142857142857,
  0.28311812529890007,
  0.2825350749879052,
  0.2801366520253783,
  0.2774792379091353],
 'pr': [0.2507621951219512,
  0.26200162733930027,
  0.2660472972972973,
  0.2761989342806394,
  0.28664192949907236,
  0.2897998093422307,
  0.2924901185770751,
  0.29554655870445345,
  0.2958762886597938,
  0.29338842975206614],
 'rec': [0.3049119555143652,
  0.29842446709916587,
  0.2919369

Проверка, что токены паддинга игнорируются

In [97]:
loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
print(loss_fct.ignore_index)  # должен быть -100

-100


In [98]:
batch['labels'][0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

In [101]:
batch['input_ids'][0]

tensor([  101, 16752, 14669,  1158,  2197,  1155,  1189,  1146,  1111,  4911,
          117,  1509, 13193,  1138,  1151,  1189,   106,   106,   108, 17607,
        11613,  1183,   102], device='cuda:0')

##### 3.1.2  Resampling methods

Отберем топ 25% примеров из conll по схожести с wnut с помощью бинарного классификатора, затем сделаем fine-tuning модели на этих топ 25 % примерах

In [19]:
from datasets import concatenate_datasets
importlib.reload(model_utils)

columns = ["input_ids", "token_type_ids", "attention_mask", "labels", "domain_label"]

conll_labeled = conll["train"].map(lambda x: {"domain_label": int(0)})
columns_to_remove = [col for col in conll_labeled.column_names if col not in columns]
conll_labeled = conll_labeled.remove_columns(columns_to_remove)

wnut_labeled = wnut["train"].map(lambda x: {"domain_label": int(1)})
columns_to_remove = [col for col in wnut_labeled.column_names if col not in columns]
wnut_labeled = wnut_labeled.remove_columns(columns_to_remove)

combined = concatenate_datasets([conll_labeled, wnut_labeled]) 

In [20]:
# делим combined на k фолдов, учим k классификаторов на остальных k-1 фолдах, добавляем в combined wnut_score
# для каждого примера получаем эмбеддинг предложения с помощью [CLF] токена BERTа
# для каждого примера из тестового фолда добавляем wnut_score
# затем из combined отбираем топ 25 % примеров по этому скору

# сначала получить эмбеддинги всех предложений
# затем сделать k классификаторов по типу leave-one-out
# классификатор X: [batch_size, emb_len] -> wnut_scores: [batch_size]

importlib.reload(model_utils)
importlib.reload(data_prep)

model_name = "dslim/bert-base-NER"
model = AutoModelForTokenClassification.from_pretrained(model_name)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataloader = torch.utils.data.DataLoader(
    combined,
    batch_size=32,
    collate_fn=data_prep.PadSequence(['input_ids', 'token_type_ids', 'attention_mask', 'labels']),
)

all_embeddings = []

for batch in tqdm(dataloader, desc="Extracting embeddings"):
    embeddings = model_utils.get_sentence_embeddings(model, batch)  # [batch_size, emb_size]
    all_embeddings.append(embeddings)

all_embeddings = np.vstack(all_embeddings)  # [num_rows, emb_size]

combined = combined.add_column("cls_embedding", all_embeddings.tolist())

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Extracting embeddings: 100%|██████████████████████████████████████████████████████████| 545/545 [00:34<00:00, 15.83it/s]


In [21]:
combined

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'domain_label', 'cls_embedding'],
    num_rows: 17435
})

In [22]:
combined['cls_embedding'].shape

torch.Size([17435, 768])

In [23]:
# теперь добавим wnut_score в combined для дальнейшего отбора

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
wnut_scores = np.zeros(len(combined))

indices = np.arange(len(combined))
for train_idxs, test_idxs in kf.split(indices):
    train_examples = combined.select(train_idxs)
    test_examples = combined.select(test_idxs)

    train_dataloader = torch.utils.data.DataLoader(
        train_examples, batch_size=32,
        collate_fn=data_prep.PadSequence(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    )
    test_dataloader = torch.utils.data.DataLoader(
        test_examples, batch_size=32,
        collate_fn=data_prep.PadSequence(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    )

    X_train, Y_train = [], []

    for batch in tqdm(train_dataloader, desc="Train embeddings"):
        embs = torch.stack(batch["cls_embedding"]).cpu().numpy()
        domains = torch.stack(batch['domain_label']).cpu().numpy()
        X_train.append(embs)
        Y_train.append(domains)
    X_train = np.concatenate(X_train)
    Y_train = np.concatenate(Y_train)

    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, Y_train)

    X_test = []
    for batch in tqdm(test_dataloader, desc="Test embeddings"):
        embs = torch.stack(batch["cls_embedding"]).cpu().numpy()
        X_test.append(embs)
    X_test = np.concatenate(X_test)
    probas = clf.predict_proba(X_test)[:, 1]

    wnut_scores[test_idxs] = probas

combined = combined.add_column("wnut_score", wnut_scores.tolist())


Test embeddings: 100%|███████████████████████████████████████████████████████████████| 109/109 [00:00<00:00, 232.10it/s]


In [24]:
combined

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'domain_label', 'cls_embedding', 'wnut_score'],
    num_rows: 17435
})

In [25]:
# отбираем топ 25 % объектов из conll по схожести с датасетом с wnut

conll_subset = combined.filter(lambda x: x["domain_label"] == 0)

conll_sorted = conll_subset.sort("wnut_score", reverse=True) 

# построить гистограмму с wnut_score

top_25_count = int(0.25 * len(conll_sorted))
top_25_conll = conll_sorted.select(range(top_25_count))

top_25_conll = top_25_conll.remove_columns([
    col for col in top_25_conll.column_names if col not in ['input_ids', 'token_type_ids', 'attention_mask', 'labels']
])

Теперь есть топ 25 % примеров из CONLL для файн-тюна BERTа

In [26]:
top_25_conll

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 3510
})

In [27]:
wnut['train']

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'text_labels'],
    num_rows: 3394
})

Размеры датасетов wnut['train'] и top_25_conll почти одинаковы. Посмотрим что будет, если файн-тюнить классификатор на top_25_conll с теми же гиперпараметрами, что и для файн-тюна на wnut['train']

In [28]:
model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for param in model_ft.bert.parameters():
    param.requires_grad = False
trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
# optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)
optimizer = torch.optim.Adam(trainable_params, lr=2e-5, betas=(0.9, 0.999), eps=1e-08)

num_epochs = 10
warmup_factor = 0.1

ft_train_dataset = top_25_conll
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(warmup_factor * num_training_steps),
    num_training_steps=num_training_steps,
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:03<00:00, 22.80it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 220/220 [00:06<00:00, 34.15it/s]


...

Epoch: 8	 Train: 0.13619176188314502 	 Test: 0.32397360957147164



Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 220/220 [00:06<00:00, 33.64it/s]


Epoch: 9	 Train: 0.1250704107636755 	 Test: 0.3224452372042485



Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 220/220 [00:06<00:00, 33.66it/s]


Epoch: 10	 Train: 0.12176169727674939 	 Test: 0.32207260588988845

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.09 (-0.66)  0.08 (-0.76)  0.08 (-0.71)  0.05 (-0.40)   
MISC          0.03 (-0.55)  0.10 (-0.60)  0.05 (-0.58)  0.03 (-0.14)   
ORG           0.01 (-0.70)  0.02 (-0.80)  0.02 (-0.74)  0.06 (-0.10)   
PER           0.02 (-0.34)  0.03 (-0.55)  0.03 (-0.42)  0.03 (-0.27)   
micro avg     0.04 (-0.54)  0.05 (-0.69)  0.04 (-0.61)  0.04 (-0.20)   
macro avg     0.04 (-0.56)  0.06 (-0.68)  0.04 (-0.62)  0.05 (-0.22)   
weighted avg  0.04 (-0.57)  0.05 (-0.69)  0.04 (-0.62)  0.04 (-0.22)   

                    rec_ft         f1_ft  
LOC           0.14 (-0.37)  0.08 (-0.40)  
MISC          0.06 (-0.16)  0.04 (-0.15)  
ORG           0.05 (-0.25)  0.05 (-0.16)  
PER           0.03 (-0.27)  0.03 (-0.27)  
micro avg     0.06 (-0.25)  0.05 (-0.22)  
macro avg     0.07 (-0.26)  0.05 (-0.24)  
weighted avg  0.06 (-0.25)  0.05 (-0.23)  

metric |   

In [29]:
ft_metrics_after

{'train_loss': [1.2603430168195204,
  0.9448994383215904,
  0.673108392750675,
  0.45281158448620273,
  0.29694188260896637,
  0.2057333342730999,
  0.15927771781994537,
  0.13619176188314502,
  0.1250704107636755,
  0.12176169727674939],
 'test_loss': [0.5994891316434483,
  0.5128830778010097,
  0.4423470243259712,
  0.38964642703901103,
  0.3548698776666029,
  0.3360696185702159,
  0.3276628636651569,
  0.32397360957147164,
  0.3224452372042485,
  0.32207260588988845],
 'f1': [0.27060270602706027,
  0.26825266611977033,
  0.2708247845711941,
  0.26813365933170336,
  0.21632024634334107,
  0.1366691560866318,
  0.0848529959062151,
  0.058245083207261725,
  0.05151864667435602,
  0.04833141542002302],
 'pr': [0.2426470588235294,
  0.24061810154525387,
  0.24300441826215022,
  0.23927272727272728,
  0.18499012508229098,
  0.11444652908067542,
  0.0708955223880597,
  0.049201277955271565,
  0.04402102496714849,
  0.0412303664921466],
 'rec': [0.30583873957367935,
  0.3030583873957368,
  

Видно, что максимальное улучшение метрик хуже, чем просто для файн-тюна с теми же гиперпараметрами

#### 3.2 Unsupervised DA

##### 3.2.1 Proxy-labels methods

До этого использовались размеченные данные. Теперь будем отбирать наиболее уверенные с точки зрения пре-тренированной модели примеры, делать предсказания и дообучать на этих прокси-лейблах, чтобы перенести знания из целевого домена.

In [30]:
# лучше сделать рестарт ядра перед запуском ячеек ниже
# ниже повторная предобработка данных
import datasets
import typing as tp
import inspect

conll = datasets.load_dataset("conll2003") # DatasetDict
wnut = datasets.load_dataset("wnut_17")

CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names # всего 9 тегов
WNUT_NER_TAGS = wnut['train'].features['ner_tags'].feature.names

label_mapping = {
    'O': 'O',
    'B-location': 'B-LOC',
    'I-location': 'I-LOC',
    'B-group': 'B-ORG',
    'B-corporation': 'B-ORG',
    'B-person': 'B-PER',
    'B-creative-work': 'B-MISC',
    'B-product': 'B-MISC',
    'I-person': 'I-PER',
    'I-creative-work': 'I-MISC',
    'I-corporation': 'I-ORG',
    'I-group': 'I-ORG',
    'I-product': 'I-MISC'
}

label2id_conll= {'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

labelindexmapping = {WNUT_NER_TAGS.index(k):CONLL_NER_TAGS.index(v) for k, v in label_mapping.items()}

import data_prep

converted_wnut = wnut.map(lambda x: data_prep.convert_label_sequence(x, labelindexmapping))

import numpy as np
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline

import torch 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

from tqdm.auto import tqdm

from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)

import importlib
import data_prep

importlib.reload(data_prep)

test_sentence = "His name is Jerry Abrahamson"
test_example = {"tokens": test_sentence.split(" "), "ner_tags": [0, 0, 0, 1, 2]}
test_result = data_prep.tokenize_and_preserve_tags(test_example, tokenizer, label2id_conll)

assert tokenizer.decode(test_result['input_ids']) == '[CLS] His name is Jerry Abrahamson [SEP]'

                                     #CLS     His  name is    Jerry    Abraham   ##son      SEP
assert test_result['text_labels'] == ['O'] + ["O", "O", "O", "B-PER", "I-PER",  "I-PER"] + ["O"]

conll = conll.map(lambda x: data_prep.tokenize_and_preserve_tags(x, tokenizer, label2id_conll))

wnut = converted_wnut
wnut = wnut.map(lambda x: data_prep.tokenize_and_preserve_tags(x, tokenizer, label2id_conll))

conll.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], output_all_columns=True)
wnut.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], output_all_columns=True)

input_keys = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']
batch_size = 32
wnut_train_dataloader = torch.utils.data.DataLoader(wnut["train"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
wnut_test_dataloader = torch.utils.data.DataLoader(wnut["test"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
conll_train_dataloader = torch.utils.data.DataLoader(conll["train"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
conll_test_dataloader = torch.utils.data.DataLoader(conll["test"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))

In [31]:
model_name = "dslim/bert-base-NER"
model = AutoModelForTokenClassification.from_pretrained(model_name)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
batch = next(iter(wnut_train_dataloader))
with torch.no_grad():
    outputs = model(input_ids=batch["input_ids"],
                 token_type_ids=batch["token_type_ids"],
                 attention_mask=batch["attention_mask"],
                 labels=batch["labels"], return_dict=True)

print(outputs.logits.shape)
attn_mask = batch['attention_mask']
probs = torch.softmax(outputs.logits, dim=-1)
max_probs = probs.max(dim=-1).values

confidences = []
for i in range(max_probs.shape[0]):
    mask = attn_mask[i].bool()
    conf = max_probs[i][mask].mean().item()
    confidences.append(conf)

print(confidences)

torch.Size([32, 47, 9])
[0.9770017266273499, 0.9972400665283203, 0.9488642811775208, 0.9996575713157654, 0.9515469074249268, 0.9970041513442993, 0.999862551689148, 0.999904215335846, 0.9905007481575012, 0.9993622303009033, 0.989292562007904, 0.9799342751502991, 0.9869406819343567, 0.9998921751976013, 0.9998874068260193, 0.9990612864494324, 0.9570358991622925, 0.9998361468315125, 0.9946780204772949, 0.9996030926704407, 0.9914760589599609, 0.9993963241577148, 0.9936320781707764, 0.999690592288971, 0.999440610408783, 0.9997891187667847, 0.9997739791870117, 0.9849514365196228, 0.9998698234558105, 0.9916747808456421, 0.9861404895782471, 0.9998873472213745]


In [33]:
from tqdm.auto import tqdm

model = model.to(device)
model.eval()

all_confidences = []

with torch.no_grad():
    for batch in tqdm(wnut_train_dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.softmax(outputs.logits, dim=-1)             # [B, T, C]
        max_probs = probs.max(dim=-1).values                      # [B, T]

        for i in range(input_ids.size(0)):
            real_token_mask = attention_mask[i].bool()            # [T]
            conf = max_probs[i][real_token_mask].mean().item()
            all_confidences.append(conf)

  0%|          | 0/107 [00:00<?, ?it/s]

In [34]:
# убедимся, что длина совпадает
assert len(all_confidences) == len(wnut["train"])

# добавим как новое поле
wnut["train"] = wnut["train"].add_column("confidence", all_confidences)

In [35]:
confidences = np.array(all_confidences)
threshold = np.percentile(confidences, 50)

filtered_wnut = wnut["train"].filter(lambda example: example["confidence"] >= threshold)

Датасет для файн-тюна готов.

Как обычно загрузим базовую модель (без файн-тюна) и будем сравнивать результаты с ней

In [36]:
filtered_wnut

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'text_labels', 'confidence'],
    num_rows: 1697
})

In [37]:
# добавляем прокси-метки в датасет
filtered_wnut_dataloader = torch.utils.data.DataLoader(filtered_wnut,        
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))

# снова получим прокси-лейблы для топ 50 процентов самых уверенных примеров из wnut_train
# получаем прокси-лейблы
ner = NamedEntityPredictor(model_ft, tokenizer, id2label=id2label_conll)
text_proxy_labels = []
proxy_labels = []

for batch in filtered_wnut_dataloader:
    text_labels_batch = ner.predict(batch)['predicted_labels']  # list[list[str]]
    text_proxy_labels.extend(text_labels_batch)

    # переводим текстовые метки в индексы
    index_batch = [
        [label2id_conll[label] for label in seq]
        for seq in text_labels_batch
    ]
    proxy_labels.extend(index_batch)

# создаем новый датасет с заменой labels и text_labels
filtered_wnut_proxy = filtered_wnut.remove_columns(['labels', 'text_labels']) \
                                   .add_column('labels', proxy_labels) \
                                   .add_column('text_labels', text_proxy_labels)

In [38]:
model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for param in model_ft.bert.parameters():
    param.requires_grad = False

# for name, param in model_ft.named_parameters():
#     if name.startswith("bert.embeddings") or name.startswith("bert.encoder.layer.0") or \
#        name.startswith("bert.encoder.layer.1") or name.startswith("bert.encoder.layer.2") or \
#        name.startswith("bert.encoder.layer.3") or name.startswith("bert.encoder.layer.4") or \
#        name.startswith("bert.encoder.layer.5") or name.startswith("bert.encoder.layer.6") or \
#        name.startswith("bert.encoder.layer.7"):
#         param.requires_grad = False
    
trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
# optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)
optimizer = torch.optim.Adam(trainable_params, lr=2e-5, betas=(0.9, 0.999), eps=1e-08)

num_epochs = 10
warmup_factor = 0.1

ft_train_dataset = filtered_wnut_proxy
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(warmup_factor * num_training_steps),
    num_training_steps=num_training_steps,
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:03<00:00, 22.93it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:03<00:00, 33.72it/s]


...

Epoch: 8	 Train: 0.1018688498847143 	 Test: 0.5897175774530128



Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:03<00:00, 33.18it/s]


Epoch: 9	 Train: 0.09991647766100104 	 Test: 0.5874280808148561



Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 107/107 [00:03<00:00, 33.18it/s]


Epoch: 10	 Train: 0.09926836100781214 	 Test: 0.5866804956286041

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.75 (+0.00)  0.84 (+0.00)  0.79 (+0.00)  0.43 (-0.02)   
MISC          0.56 (-0.02)  0.70 (+0.00)  0.62 (-0.01)  0.15 (-0.02)   
ORG           0.72 (+0.01)  0.81 (-0.01)  0.76 (+0.00)  0.19 (+0.03)   
PER           0.37 (+0.01)  0.57 (-0.01)  0.45 (+0.00)  0.32 (+0.02)   
micro avg     0.59 (+0.01)  0.74 (+0.00)  0.65 (+0.00)  0.25 (+0.01)   
macro avg     0.60 (+0.00)  0.73 (-0.01)  0.66 (+0.00)  0.27 (+0.00)   
weighted avg  0.61 (+0.00)  0.74 (+0.00)  0.66 (+0.00)  0.27 (+0.01)   

                    rec_ft         f1_ft  
LOC           0.52 (+0.01)  0.47 (-0.01)  
MISC          0.22 (+0.00)  0.18 (-0.01)  
ORG           0.27 (-0.03)  0.22 (+0.01)  
PER           0.28 (-0.02)  0.30 (+0.00)  
micro avg     0.30 (-0.01)  0.27 (+0.00)  
macro avg     0.32 (-0.01)  0.29 (+0.00)  
weighted avg  0.30 (-0.01)  0.28 (+0.00)  

metric |    

In [39]:
ft_metrics_after

{'train_loss': [0.15514242801288683,
  0.1428793407699189,
  0.13230941449368164,
  0.1233204874306891,
  0.1158785819343648,
  0.1098267687647821,
  0.10517439774126598,
  0.1018688498847143,
  0.09991647766100104,
  0.09926836100781214],
 'test_loss': [0.6412174041256492,
  0.6313148220012217,
  0.6219541011033235,
  0.6126955077971941,
  0.6047563201483385,
  0.5987705165221368,
  0.5931387023057466,
  0.5897175774530128,
  0.5874280808148561,
  0.5866804956286041],
 'f1': [0.27412008281573497,
  0.2732971165900543,
  0.2745263157894737,
  0.27184466019417475,
  0.27299703264094954,
  0.27091295116772823,
  0.2716468590831918,
  0.2711432214194645,
  0.27125850340136054,
  0.2713738834538494],
 'pr': [0.24775449101796407,
  0.24885844748858446,
  0.2515432098765432,
  0.2496124031007752,
  0.2515625,
  0.25,
  0.2505873140172279,
  0.25039246467817894,
  0.25058915946582877,
  0.2507861635220126],
 'rec': [0.30676552363299353,
  0.3030583873957368,
  0.30213160333642264,
  0.2984244

Если заморозить первые 8 слоев BERTа

In [44]:
model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for param in model_ft.bert.parameters():
    param.requires_grad = False

# for name, param in model_ft.named_parameters():
#     if name.startswith("bert.embeddings") or name.startswith("bert.encoder.layer.0") or \
#        name.startswith("bert.encoder.layer.1") or name.startswith("bert.encoder.layer.2") or \
#        name.startswith("bert.encoder.layer.3") or name.startswith("bert.encoder.layer.4") or \
#        name.startswith("bert.encoder.layer.5") or name.startswith("bert.encoder.layer.6") or \
#        name.startswith("bert.encoder.layer.7"):
#         param.requires_grad = False
    
trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
# optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)
optimizer = torch.optim.Adam(trainable_params, lr=2e-5, betas=(0.9, 0.999), eps=1e-08)
# optimizer = AdamW(trainable_params, lr=2e-5, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.01)

num_epochs = 10
warmup_factor = 0.1

ft_train_dataset = filtered_wnut_proxy
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(warmup_factor * num_training_steps),
    num_training_steps=num_training_steps,
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:03<00:00, 22.75it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:04<00:00, 22.58it/s]


...

Epoch: 8	 Train: 0.014924626295736591 	 Test: 0.5142114935097871



Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:04<00:00, 22.42it/s]


Epoch: 9	 Train: 0.014126561844566942 	 Test: 0.5164081595930052



Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 107/107 [00:04<00:00, 22.39it/s]


Epoch: 10	 Train: 0.01387225919692542 	 Test: 0.5179275454562388

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.19 (-0.56)  0.34 (-0.50)  0.25 (-0.54)  0.02 (-0.43)   
MISC          0.19 (-0.39)  0.69 (-0.01)  0.30 (-0.33)  0.10 (-0.07)   
ORG           0.14 (-0.57)  0.10 (-0.72)  0.12 (-0.64)  0.01 (-0.15)   
PER           0.00 (-0.36)  0.00 (-0.58)  0.00 (-0.45)  0.00 (-0.30)   
micro avg     0.14 (-0.44)  0.22 (-0.52)  0.17 (-0.48)  0.05 (-0.19)   
macro avg     0.13 (-0.47)  0.28 (-0.46)  0.17 (-0.49)  0.03 (-0.24)   
weighted avg  0.12 (-0.49)  0.22 (-0.52)  0.15 (-0.51)  0.03 (-0.23)   

                    rec_ft         f1_ft  
LOC           0.07 (-0.44)  0.03 (-0.45)  
MISC          0.24 (+0.02)  0.14 (-0.05)  
ORG           0.00 (-0.30)  0.01 (-0.20)  
PER           0.00 (-0.30)  0.00 (-0.30)  
micro avg     0.07 (-0.24)  0.06 (-0.21)  
macro avg     0.08 (-0.25)  0.05 (-0.24)  
weighted avg  0.07 (-0.24)  0.04 (-0.24)  

metric |    

In [45]:
ft_metrics_after

{'train_loss': [0.1182837890141217,
  0.052627770420572956,
  0.03533619093419288,
  0.02697076203103534,
  0.0219786444958517,
  0.018449030451941146,
  0.016246403792163654,
  0.014924626295736591,
  0.014126561844566942,
  0.01387225919692542],
 'test_loss': [0.5857875069350372,
  0.47295676907639445,
  0.47353565012231286,
  0.4831461395010536,
  0.4924837188956178,
  0.5057527014502773,
  0.5116289835653187,
  0.5142114935097871,
  0.5164081595930052,
  0.5179275454562388],
 'f1': [0.2609078889378581,
  0.18713450292397663,
  0.13495864170657382,
  0.10855949895615866,
  0.09551255660765746,
  0.0813615608136156,
  0.07434640522875817,
  0.06874241811564902,
  0.06360708534621577,
  0.06131504638967325],
 'pr': [0.24873949579831933,
  0.18181818181818182,
  0.1272577996715928,
  0.09878419452887538,
  0.08592592592592592,
  0.07368421052631578,
  0.0664718772826881,
  0.06097560975609756,
  0.056227758007117434,
  0.054285714285714284],
 'rec': [0.2743280815569972,
  0.19277108433

In [42]:
# сравним настоящие и прокси метки на одном примере
filtered_wnut_dataloader_proxy = torch.utils.data.DataLoader(filtered_wnut_proxy,        
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))

print(next(iter(filtered_wnut_dataloader))['labels'][0])
print(next(iter(filtered_wnut_dataloader_proxy))['labels'][0])

tensor([   0,    0,    0,    0,    0,    0,    0,    3,    4,    4,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100], device='cuda:0')
tensor([   0,    0,    1,    2,    2,    2,    0,    1,    2,    2,    0,    0,
           0,    7,    8,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100, -100,
        -100], device='cuda:0')


Сравнение с обычным файн-тюном на всем трейне wnut при тех же гиперпараметрах

In [47]:
# берем не топ 50 процентов по уверенности модели, а просто 50 процентов рандомных примеров из wnut train
import random

np.random.seed(42)
random_indices = random.sample(range(len(wnut["train"])), len(filtered_wnut))
wnut_train_random_subs = [wnut["train"][i] for i in random_indices]

model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for param in model_ft.bert.parameters():
    param.requires_grad = False

# for name, param in model_ft.named_parameters():
#     if name.startswith("bert.embeddings") or name.startswith("bert.encoder.layer.0") or \
#        name.startswith("bert.encoder.layer.1") or name.startswith("bert.encoder.layer.2") or \
#        name.startswith("bert.encoder.layer.3") or name.startswith("bert.encoder.layer.4") or \
#        name.startswith("bert.encoder.layer.5") or name.startswith("bert.encoder.layer.6") or \
#        name.startswith("bert.encoder.layer.7"):
#         param.requires_grad = False
    
trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
# optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)
optimizer = torch.optim.Adam(trainable_params, lr=2e-5, betas=(0.9, 0.999), eps=1e-08)

num_epochs = 10
warmup_factor = 0.1

ft_train_dataset = wnut_train_random_subs
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(warmup_factor * num_training_steps),
    num_training_steps=num_training_steps,
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:03<00:00, 21.45it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:03<00:00, 29.74it/s]


...

Epoch: 8	 Train: 0.3095226195231776 	 Test: 0.49577684314162646



Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:03<00:00, 29.42it/s]


Epoch: 9	 Train: 0.3058671420701196 	 Test: 0.49096254066184714



Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 107/107 [00:03<00:00, 29.40it/s]


Epoch: 10	 Train: 0.30464231571025935 	 Test: 0.4893680982015751

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.76 (+0.01)  0.84 (+0.00)  0.80 (+0.01)  0.49 (+0.04)   
MISC          0.57 (-0.01)  0.69 (-0.01)  0.63 (+0.00)  0.17 (+0.00)   
ORG           0.72 (+0.01)  0.81 (-0.01)  0.76 (+0.00)  0.19 (+0.03)   
PER           0.38 (+0.02)  0.59 (+0.01)  0.46 (+0.01)  0.33 (+0.03)   
micro avg     0.59 (+0.01)  0.74 (+0.00)  0.66 (+0.01)  0.27 (+0.03)   
macro avg     0.61 (+0.01)  0.73 (-0.01)  0.66 (+0.00)  0.29 (+0.02)   
weighted avg  0.62 (+0.01)  0.74 (+0.00)  0.67 (+0.01)  0.28 (+0.02)   

                    rec_ft         f1_ft  
LOC           0.51 (+0.00)  0.50 (+0.02)  
MISC          0.19 (-0.03)  0.18 (-0.01)  
ORG           0.27 (-0.03)  0.23 (+0.02)  
PER           0.28 (-0.02)  0.30 (+0.00)  
micro avg     0.29 (-0.02)  0.28 (+0.01)  
macro avg     0.31 (-0.02)  0.30 (+0.01)  
weighted avg  0.29 (-0.02)  0.28 (+0.00)  

metric |    

In [48]:
ft_metrics_after

{'train_loss': [0.4180905382349112,
  0.3914361883685968,
  0.36940671482654375,
  0.35110347286284527,
  0.33629396404618417,
  0.32464878783326284,
  0.31581655430181005,
  0.3095226195231776,
  0.3058671420701196,
  0.30464231571025935],
 'test_loss': [0.6289433943636623,
  0.5983543199153594,
  0.571914682601705,
  0.5492657277687096,
  0.5305876625172886,
  0.5156259777737252,
  0.5040447684349837,
  0.49577684314162646,
  0.49096254066184714,
  0.4893680982015751],
 'f1': [0.275290215588723,
  0.2753684210526316,
  0.27630453378956377,
  0.27999999999999997,
  0.2789886660854403,
  0.27880386983289357,
  0.2800177226406734,
  0.2803571428571428,
  0.2789450156459544,
  0.27931960608773504],
 'pr': [0.2490622655663916,
  0.2523148148148148,
  0.2565528196981732,
  0.26371826371826373,
  0.26337448559670784,
  0.26527196652719665,
  0.26825127334465193,
  0.2704565030146425,
  0.2694300518134715,
  0.2701298701298701],
 'rec': [0.3076923076923077,
  0.3030583873957368,
  0.29935125

Файн-тюн на истинных метках ожидаемо оказался лучше файн-тюна на прокси-метках

##### 3.2.2 Unsupervised pretraining


In [1]:
import json

data = []
with open('reddit_sample.json', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

from datasets import Dataset
reddit_dataset = Dataset.from_list(data)
reddit_dataset= reddit_dataset.rename_column("words", "tokens")

AttributeError: type object 'Dataset' has no attribute 'from_list'

Аналогично 3.1.2 отберем 100 000 примеров, наиболее похожих на wnut['train'] и дообучим BERT на задачу masked LM.

In [ ]:
wnut['train']

In [122]:
columns_to_keep = ['tokens'] # называется токены, но в датасете это просто списки слов
columns_to_remove = [col for col in wnut['train'].column_names if col not in columns_to_keep]

wnut_train_new = wnut['train'].remove_columns(columns_to_remove)

In [123]:
wnut_train_new

Dataset({
    features: ['tokens'],
    num_rows: 3394
})

In [124]:
reddit_dataset

Dataset({
    features: ['tokens'],
    num_rows: 500000
})

In [125]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

def tokenize_batch(batch):
    return tokenizer(
        batch["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding=False,
        max_length=512
    )

reddit_dataset = reddit_dataset.map(
    tokenize_batch,
    batched=True,
    remove_columns=[]
)

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [127]:
wnut_train_new = wnut_train_new.map(
    tokenize_batch,
    batched=True,
    remove_columns=[]
)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

In [128]:
reddit_dataset = reddit_dataset.map(lambda x: {"domain_label": int(0)})
wnut_train_new = wnut_train_new.map(lambda x: {"domain_label": int(1)})

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

In [129]:
reddit_dataset

Dataset({
    features: ['tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'domain_label'],
    num_rows: 500000
})

In [130]:
wnut_train_new

Dataset({
    features: ['tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'domain_label'],
    num_rows: 3394
})

In [131]:
from datasets import concatenate_datasets

combined = concatenate_datasets([reddit_dataset, wnut_train_new]) 

In [132]:
combined.set_format("torch")

In [134]:
combined['domain_label'][500_001]

tensor(1)

In [135]:
from transformers import BertModel
import torch
from tqdm.auto import tqdm

tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
model = BertModel.from_pretrained("bert-base-cased")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [136]:
for i, ex in enumerate(combined):
    for key in ["input_ids", "attention_mask", "token_type_ids"]:
        if ex.get(key) is None:
            print(f"Пропущено значение в примере {i}: {key} = None")

In [137]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import numpy as np

import data_prep
from data_prep import PadSequence

collate_fn = PadSequence(['input_ids','token_type_ids','attention_mask'])

dataloader = torch.utils.data.DataLoader(
    combined,
    batch_size=32,
    shuffle=False,
    collate_fn=collate_fn
)

batch = next(iter(dataloader))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

all_embeddings = []

batch

for batch in tqdm(dataloader, desc="Extracting embeddings"):
    batch = {k: batch[k].to(device) for k in ['input_ids','token_type_ids','attention_mask']}
    with torch.no_grad():
        outputs=model(**batch)
        cls_emb = model(**batch).last_hidden_state[:, 0] # (batch_size, emb_size=768)
    all_embeddings.append(cls_emb.cpu())

all_embeddings = torch.cat(all_embeddings).numpy()

Extracting embeddings:   0%|          | 0/15732 [00:00<?, ?it/s]

In [138]:
combined = combined.add_column("cls_embedding", all_embeddings.tolist())

In [139]:
combined

Dataset({
    features: ['tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'domain_label', 'cls_embedding'],
    num_rows: 503394
})

In [140]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

k = 5
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
wnut_scores = np.zeros(len(combined))
X = combined["cls_embedding"].cpu().numpy()
Y = combined["domain_label"].cpu().numpy()

indices = np.arange(len(combined))
for i, (train_idxs, test_idxs) in enumerate(tqdm(kf.split(X, Y), total=k, desc="Training classifiers")):
    X_train, X_test = X[train_idxs], X[test_idxs]
    Y_train = Y[train_idxs]
    
    clf = LogisticRegression(max_iter=1000, class_weight="balanced")
    clf.fit(X_train, Y_train)
    probas = clf.predict_proba(X_test)[:, 1]
    
    wnut_scores[test_idxs] = probas
    
combined = combined.add_column("wnut_score", wnut_scores.tolist())

Training classifiers:   0%|          | 0/5 [00:00<?, ?it/s]

In [141]:
# отбираем топ 20 % объектов из reddit_dataset по схожести с датасетом с wnut

reddit_subset = combined.filter(lambda x: x["domain_label"] == 0)

reddit_sorted = reddit_subset.sort("wnut_score", reverse=True) 

# построить гистограмму с wnut_score

top_20_count = int(0.20 * len(reddit_sorted))
top_20_reddit = reddit_sorted.select(range(top_20_count))

# top_20_reddit = top_20_reddit.remove_columns([
#     col for col in top_20_reddit.column_names if col not in ['input_ids', 'token_type_ids', 'attention_mask']
# ])

Filter:   0%|          | 0/503394 [00:00<?, ? examples/s]

In [142]:
sel_examples = top_20_reddit['tokens']

import json
with open("sel_examples.json", "w", encoding="utf-8") as f:
    json.dump(sel_examples, f, ensure_ascii=False, indent=2)

In [119]:
top_20_reddit['input_ids'][0]

tensor([  101,  1122,   134, 27322, 17482,  2340,   119,  8916,   102])

Теперь есть топ 20 % из 500 000 примеров для файн-тюна самого BERTа на задаче masked LM.

Код для файн-тюнинга BERTа на задаче MLM лежит в ноутбуке mlm. 

In [1]:
import datasets
import typing as tp
import inspect
import data_prep
import numpy as np
import sklearn
import torch 
from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)
from datasets import load_dataset, load_from_disk
import torch
from tqdm.auto import tqdm
from transformers import AdamW, BertTokenizer, BertForMaskedLM, set_seed, BertTokenizerFast
import pandas as pd
import warnings

import data_prep
import importlib
importlib.reload(data_prep)

conll = load_from_disk("datasets/conll2003")
wnut = load_from_disk("datasets/wnut_17")

CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names # всего 9 тегов
WNUT_NER_TAGS = wnut['train'].features['ner_tags'].feature.names

label_mapping = {
    'O': 'O',
    'B-location': 'B-LOC',
    'I-location': 'I-LOC',
    'B-group': 'B-ORG',
    'B-corporation': 'B-ORG',
    'B-person': 'B-PER',
    'B-creative-work': 'B-MISC',
    'B-product': 'B-MISC',
    'I-person': 'I-PER',
    'I-creative-work': 'I-MISC',
    'I-corporation': 'I-ORG',
    'I-group': 'I-ORG',
    'I-product': 'I-MISC'
}

labelindexmapping = {WNUT_NER_TAGS.index(k):CONLL_NER_TAGS.index(v) for k, v in label_mapping.items()}

converted_wnut = wnut.map(lambda x: data_prep.convert_label_sequence(x, labelindexmapping))

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model_name = "dslim/bert-base-NER"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

set_seed(42)

label2id_conll= {'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8
}

id2label_conll = {v : k for k, v in label2id_conll.items()}

test_sentence = "His name is Jerry Abrahamson"
test_example = {"tokens": test_sentence.split(" "), "ner_tags": [0, 0, 0, 1, 2]}
test_result = data_prep.tokenize_and_preserve_tags(test_example, tokenizer, label2id_conll)

assert tokenizer.decode(test_result['input_ids']) == '[CLS] His name is Jerry Abrahamson [SEP]'

                                     #CLS     His  name is    Jerry    Abraham   ##son      SEP
assert test_result['text_labels'] == ['O'] + ["O", "O", "O", "B-PER", "I-PER",  "I-PER"] + ["O"]

conll = conll.map(lambda x: data_prep.tokenize_and_preserve_tags(x, tokenizer, label2id_conll))

wnut = converted_wnut
wnut = wnut.map(lambda x: data_prep.tokenize_and_preserve_tags(x, tokenizer, label2id_conll))

conll.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], output_all_columns=True)
wnut.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], output_all_columns=True)

input_keys = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']
batch_size = 16
wnut_train_dataloader = torch.utils.data.DataLoader(wnut["train"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
wnut_test_dataloader = torch.utils.data.DataLoader(wnut["test"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
conll_train_dataloader = torch.utils.data.DataLoader(conll["train"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
conll_test_dataloader = torch.utils.data.DataLoader(conll["test"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))

/home/daniil_tomlv/miniconda3/envs/lmenv/lib/python3.10/site-packages/transformers/data/metrics/__init__.py:19: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.22.3)
  from scipy.stats import pearsonr, spearmanr
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
counter = {k : 0 for k in label2id_conll.keys()}
for example in wnut['train']['text_labels']:
    for label in example:
        counter[label] += 1

total=0
for count in counter.values():
    total+=count

freqs = {
    "O" : counter['O']/total*100,
    "PER" : (counter['B-PER']+counter['I-PER'])/total*100,
    "ORG" : (counter['B-ORG']+counter['I-ORG'])/total*100,
    "LOC" : (counter['B-LOC']+counter['I-LOC'])/total*100,
    "MISC" : (counter['B-MISC']+counter['I-MISC'])/total*100  
}

freqs = {k : round(v,3) for k, v in freqs.items()}

freqs

{'O': 95.293, 'PER': 1.485, 'ORG': 1.068, 'LOC': 1.125, 'MISC': 1.028}

In [3]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, get_scheduler, BertTokenizerFast
import torch
from torch.nn.utils import clip_grad_norm_ # выполняет обрезку градиентов для избежания их взрыва при обучении
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np

import importlib
import data_prep
import model_utils
importlib.reload(data_prep)
importlib.reload(model_utils)

from model_utils import train_eval_ner

In [4]:
model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for param in model_ft.bert.parameters():
    param.requires_grad = False
trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
optimizer = torch.optim.AdamW(trainable_params, lr=2e-5, betas=(0.9, 0.999), eps=1e-08)

num_epochs = 1
warmup_factor = 0.1

ft_train_dataset = wnut['train']
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(warmup_factor * num_training_steps),
    num_training_steps=num_training_steps,
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:01<00:00, 41.26it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 213/213 [00:03<00:00, 55.76it/s]


Epoch: 1	 Train: 0.41513655572569985 	 Test: 0.6117660495234124
F1: 0.27580442958629336
LR: [0.0]

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.75 (+0.00)  0.84 (+0.00)  0.79 (+0.00)  0.46 (+0.01)   
MISC          0.58 (+0.00)  0.70 (+0.00)  0.64 (+0.01)  0.18 (+0.01)   
ORG           0.71 (+0.00)  0.82 (+0.00)  0.76 (+0.00)  0.18 (+0.02)   
PER           0.37 (+0.01)  0.58 (+0.00)  0.45 (+0.00)  0.30 (+0.00)   
micro avg     0.58 (+0.00)  0.74 (+0.00)  0.65 (+0.00)  0.25 (+0.01)   
macro avg     0.60 (+0.00)  0.73 (-0.01)  0.66 (+0.00)  0.28 (+0.01)   
weighted avg  0.61 (+0.00)  0.74 (+0.00)  0.67 (+0.01)  0.26 (+0.00)   

                    rec_ft         f1_ft  
LOC           0.51 (+0.00)  0.48 (+0.00)  
MISC          0.22 (+0.00)  0.19 (+0.00)  
ORG           0.30 (+0.00)  0.22 (+0.01)  
PER           0.29 (-0.01)  0.30 (+0.00)  
micro avg     0.31 (+0.00)  0.28 (+0.01)  
macro avg     0.33 (+0.00)  0.30 (+0.01)  
weighted avg  0.31 (+0.

Если использовать continued pre-trained модель, полученную в ноутбуке mlm

In [8]:
label2id_conll

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [9]:
lr_scheduler.get_last_lr()[0]

0.0

In [7]:
from transformers import BertTokenizerFast, BertForMaskedLM, AutoModelForTokenClassification

model_name = "dslim/bert-base-NER"
mlm_model = BertForMaskedLM.from_pretrained("model")
pretrained_bert = mlm_model.bert
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
model_ft.bert = pretrained_bert
model_ft.classifier.reset_parameters()

tokenizer = AutoTokenizer.from_pretrained("model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for name, param in model_ft.named_parameters():
    if name.startswith("bert.embeddings") or name.startswith("bert.encoder.layer.0") or \
       name.startswith("bert.encoder.layer.1") or name.startswith("bert.encoder.layer.2") or \
       name.startswith("bert.encoder.layer.3") or name.startswith("bert.encoder.layer.4") or \
       name.startswith("bert.encoder.layer.5") or name.startswith("bert.encoder.layer.6") or \
       name.startswith("bert.encoder.layer.7") or name.startswith("bert.encoder.layer.8") or \
       name.startswith("bert.encoder.layer.9"):
        param.requires_grad = False

# for name, param in model_ft.bert.named_parameters():
#     param.requires_grad = False

trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
optimizer = torch.optim.AdamW(trainable_params, lr=2e-5, betas=(0.9, 0.999), eps=1e-08)

num_epochs = 50
warmup_factor = 0.1

ft_train_dataset = wnut['train']
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 32
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(warmup_factor * num_training_steps),
    num_training_steps=num_training_steps,
)

# lr_scheduler = get_scheduler(
#     name="constant",
#     optimizer=optimizer
# )

# добавляем веса
factor = 0.01
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 41/41 [00:02<00:00, 20.09it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:03<00:00, 27.62it/s]


...

Epoch: 48	 Train: 1.0969741595125644 	 Test: 1.0225994601482298
F1: 0.010040640688501076
LR: [8.88888888888889e-07]



Epoch 49: 100%|███████████████████████████████████████████████████████████████████████| 107/107 [00:03<00:00, 27.57it/s]


Epoch: 49	 Train: 1.0966147150948784 	 Test: 1.0222193627822689
F1: 0.010040640688501076
LR: [4.444444444444445e-07]



Epoch 50: 100%|███████████████████████████████████████████████████████████████████████| 107/107 [00:03<00:00, 27.61it/s]


Epoch: 50	 Train: 1.0964217536917358 	 Test: 1.022026918283323
F1: 0.010046645138141371
LR: [0.0]

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.00 (+0.00)  0.00 (-0.02)  0.00 (+0.00)  0.00 (+0.00)   
MISC          0.00 (+0.00)  0.02 (+0.00)  0.01 (+0.01)  0.01 (+0.01)   
ORG           0.04 (+0.04)  0.08 (+0.07)  0.05 (+0.05)  0.02 (+0.02)   
PER           0.00 (-0.01)  0.00 (-0.16)  0.00 (-0.03)  0.00 (-0.01)   
micro avg     0.01 (+0.00)  0.03 (-0.03)  0.02 (+0.01)  0.01 (+0.01)   
macro avg     0.01 (+0.01)  0.03 (-0.02)  0.01 (+0.00)  0.01 (+0.01)   
weighted avg  0.01 (+0.00)  0.03 (-0.03)  0.02 (+0.01)  0.01 (+0.01)   

                    rec_ft         f1_ft  
LOC           0.01 (+0.00)  0.00 (+0.00)  
MISC          0.07 (+0.05)  0.01 (+0.01)  
ORG           0.06 (+0.00)  0.03 (+0.03)  
PER           0.02 (-0.12)  0.01 (+0.00)  
micro avg     0.04 (-0.03)  0.01 (+0.01)  
macro avg     0.04 (-0.02)  0.01 (+0.01)  
weighted avg  0.04 (-0.

In [20]:
model_ft

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [6]:
# экспериментальная версия
from transformers import BertTokenizerFast, BertForMaskedLM, AutoModelForTokenClassification

model_name = "dslim/bert-base-NER"
mlm_model = BertForMaskedLM.from_pretrained("model")
pretrained_bert = mlm_model.bert
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
model_ft.bert = pretrained_bert
model_ft.classifier.reset_parameters()

tokenizer = AutoTokenizer.from_pretrained("model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for p in model_ft.bert.parameters():           
    p.requires_grad = False
for n, p in model_ft.bert.named_parameters():
    if n.startswith(("encoder.layer.8","encoder.layer.9","encoder.layer.10", "encoder.layer.11")):
        p.requires_grad = True
for p in model_ft.classifier.parameters():     
    p.requires_grad = True

head_params = [p for n, p in model_ft.named_parameters() if "classifier" in n]
bert_params = [p for n, p in model_ft.named_parameters()
               if p.requires_grad and "classifier" not in n]

optimizer = torch.optim.AdamW(
    [
        {"params": head_params, "lr": 1e-4, "weight_decay": 0.01},
        {"params": bert_params, "lr": 2e-5, "weight_decay": 0.01},
    ],
    betas=(0.9, 0.999), eps=1e-6
)

num_epochs = 50
warmup_factor = 0.1

ft_train_dataset = wnut['train']
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 32
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(warmup_factor * num_training_steps),
    num_training_steps=num_training_steps,
)

factor = 0.01
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)

fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 41/41 [00:02<00:00, 20.11it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:06<00:00, 15.86it/s]


...

Epoch 11: 100%|███████████████████████████████████████████████████████████████████████| 107/107 [00:06<00:00, 15.72it/s]


Epoch: 11	 Train: 0.2074482659020496 	 Test: 0.3756830846754516
F1: 0.018710324089542263
LR: [8.666666666666667e-05, 1.7333333333333336e-05]



Epoch 12: 100%|███████████████████████████████████████████████████████████████████████| 107/107 [00:06<00:00, 15.71it/s]


Epoch: 12	 Train: 0.20207760063931346 	 Test: 0.4037785173916235
F1: 0.01745552198724404
LR: [8.444444444444444e-05, 1.688888888888889e-05]


KeyboardInterrupt: 

In [21]:
model_ft

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [22]:
test_sentences = [
    {'text' : 'His name is Jerry Abrahamson', 'ner_tags': [0, 0, 0, 1, 2]},
    {'text' : 'London is big city', 'ner_tags' : [5, 0, 0, 0]}
]

In [23]:
idx = 0
test_sentence = test_sentences[idx]['text']

tokenizer(test_sentence)

{'input_ids': [101, 2010, 2171, 2003, 6128, 8181, 3385, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

##### Тест NER

In [24]:
import importlib
import data_prep
importlib.reload(data_prep)

idx = 0
test_sentence = test_sentences[idx]['text']
ner_tags = test_sentences[idx]['ner_tags']
test_example = {"tokens": test_sentence.split(" "), "ner_tags": ner_tags}
test_result = data_prep.tokenize_and_preserve_tags(test_example, tokenizer, label2id_conll)
print(test_result.word_ids())
print(test_result['input_ids'])
print([tokenizer.decode(token) for token in test_result['input_ids']])

test_batch = {k : [v] for k, v in test_result.items()}

import torch
from collections import defaultdict

batched_test_batch = defaultdict(list)

for key, value in test_batch.items():
    try:
        batched_test_batch[key] = torch.tensor(value)
    except ValueError:
        batched_test_batch[key] = value

for key in batched_test_batch:
    if isinstance(batched_test_batch[key], torch.Tensor):
        batched_test_batch[key] = batched_test_batch[key].to(device)

import importlib
import model_utils
importlib.reload(model_utils)

from model_utils import NamedEntityPredictor

ner_test = NamedEntityPredictor(model_ft, tokenizer, id2label_conll)
# print(f"Predicted:\t {ner_test.predict(batched_test_batch)['predicted_labels']}")
# print(f"True:\t {batched_test_batch['text_labels']}")

pred = ner_test.predict(batched_test_batch)['predicted_labels']
true = batched_test_batch['text_labels']
for pred_seq, true_seq in zip(pred, true):
    aligned = [(t, p) for t, p in zip(true_seq, pred_seq)]
    for t, p in aligned:
        print(f"True: {t:<10}  Pred: {p:<10}")


[None, 0, 1, 2, 3, 4, 4, None]
[101, 2010, 2171, 2003, 6128, 8181, 3385, 102]
['[CLS]', 'his', 'name', 'is', 'jerry', 'abraham', '##son', '[SEP]']
True: O           Pred: O         
True: O           Pred: O         
True: O           Pred: O         
True: O           Pred: O         
True: B-PER       Pred: O         
True: I-PER       Pred: O         
True: I-PER       Pred: O         
True: O           Pred: O         
